# "앱스토어 리뷰 크롤링"
> "selenium을 활용해 동적 페이지 앱스토어 리뷰 데이터 크롤링"

- toc:true
- branch: master
- badges: true
- comments: true
- author: ib choi
- categories: [Selenium, jupyter, BigQuery]

In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import string
import datetime 

from bs4 import BeautifulSoup
import requests

In [28]:
import pandas as pd
import pymysql

# 1.  데이터 크롤링

## 1-1. 동적 페이지 데이터 불러오기 (chromdriver 활용)

In [29]:
chromedriver = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver)

# 앱스토어 자란다 페이지
driver.get(
    "https://apps.apple.com/kr/app/%EC%9E%90%EB%9E%80%EB%8B%A4/id1469857402#see-all/reviews"
)

# 동적 페이지의 데이터를 모두 불러오기 위해 스크롤을 가장 아래까지 내림

prev_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # 스크롤을 화면 가장 아래로 내린다
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(0.5)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")

    if (curr_height == prev_height):
        break
    else:
        prev_height = driver.execute_script(
            "return document.body.scrollHeight")
        
curr_height


## 1-2. 개별 리뷰 박스의 ID 크롤링

In [30]:
box=driver.find_elements_by_class_name('l-column--equal-height')

box_id = []
for i in box:
    box_id.append(i.get_attribute('id'))

    
print(box_id[:3])
print("\n 리뷰 박스 개수 =", len(box_id),'개')

['ember89', 'ember91', 'ember93']

 리뷰 박스 개수 = 45 개


## 1-3. 리뷰 데이터 크롤링

앱스토어의 리뷰 데이터는 (1) 코멘트의 길이가 길거나 혹은 (2) 개발자의 답변이 있을 경우, \
개별 리뷰 박스의 '더 보기' 버튼을 클릭해 데이터를 추가로 불어와야 크롤링 가능

In [64]:
driver.find_element_by_id('ember101').find_element_by_class_name('we-clamp').text

'아이가 공부를 너무 하기 싫어해서 고민이였는데 샘들이 친구처럼 놀아주면서 하니까 학습 효과가 너무 좋네요^^ 자주 애용해야겠어요. 아들두 쌤 또 언제 오냐구 그러고… 일반 학습지보다 나은듯~. 만족합니다^^'

In [31]:
stars = driver.find_elements_by_class_name('we-star-rating--large') # 별점
date = driver.find_elements_by_class_name('we-customer-review__date') # 작성일
user = driver.find_elements_by_class_name('we-customer-review__user') # 작성자
title = driver.find_elements_by_class_name('we-customer-review__title') # 제목

In [32]:
review = []
for id in enumerate(box_id):
    
    # 데이터 로드 대기
#     time.sleep(1)
    
    # 박스에 '더 보기' 버튼이 있을 경우
    try:
        # '더 보기' 클릭
        button = driver.find_element_by_css_selector(
            f"#{id[1]} > div.we-customer-review.lockup > blockquote:nth-child(4) > button"
        )
        button.click()
        time.sleep(0.2)

        review.append(
            driver.find_element_by_class_name(
                'we-customer-review__body--modal').text) #리뷰 코멘트
        
        # 박스 닫기
        exit = driver.find_element_by_class_name('we-modal__close')
        exit.click()
        print(id, '더 보기 클릭')

    except:
        review.append(driver.find_element_by_id(f'{id[1]}').\
                      find_element_by_class_name('we-clamp').text) #리뷰 코멘트
        print(id, 'pass')

print('\n 크롤링 완료')

(0, 'ember89') pass
(1, 'ember91') pass
(2, 'ember93') pass
(3, 'ember95') 더 보기 클릭
(4, 'ember97') 더 보기 클릭
(5, 'ember99') 더 보기 클릭
(6, 'ember101') 더 보기 클릭
(7, 'ember103') 더 보기 클릭
(8, 'ember105') 더 보기 클릭
(9, 'ember107') 더 보기 클릭
(10, 'ember120') 더 보기 클릭
(11, 'ember122') 더 보기 클릭
(12, 'ember124') 더 보기 클릭
(13, 'ember126') 더 보기 클릭
(14, 'ember128') 더 보기 클릭
(15, 'ember130') 더 보기 클릭
(16, 'ember132') 더 보기 클릭
(17, 'ember134') 더 보기 클릭
(18, 'ember136') 더 보기 클릭
(19, 'ember138') 더 보기 클릭
(20, 'ember150') 더 보기 클릭
(21, 'ember152') 더 보기 클릭
(22, 'ember154') 더 보기 클릭
(23, 'ember156') 더 보기 클릭
(24, 'ember158') 더 보기 클릭
(25, 'ember160') 더 보기 클릭
(26, 'ember162') 더 보기 클릭
(27, 'ember164') 더 보기 클릭
(28, 'ember166') 더 보기 클릭
(29, 'ember168') 더 보기 클릭
(30, 'ember180') 더 보기 클릭
(31, 'ember182') 더 보기 클릭
(32, 'ember184') 더 보기 클릭
(33, 'ember186') 더 보기 클릭
(34, 'ember188') pass
(35, 'ember190') 더 보기 클릭
(36, 'ember192') 더 보기 클릭
(37, 'ember194') pass
(38, 'ember196') 더 보기 클릭
(39, 'ember198') 더 보기 클릭
(40, 'ember205') pass
(41, 'emb

# 2. 크롤링 데이터 전처리

In [33]:
title_lst = []
for i in title:
    if i.text != '개발자 답변':
        title_lst.append(i.text)

user_lst = []
for i in user:
    user_lst.append(i.text)

date_lst = []
for i in date:
    z = i.get_attribute('aria-label')
    date_lst.append(
        # str -> datetime 변환
        datetime.datetime.strptime(
            z.replace('년 ', '-').replace('월 ', '-').replace('일', ''),
            "%Y-%m-%d"))

stars_lst = []
for i in stars:
    stars_lst.append(
        # int로 변환
        int(i.get_attribute('aria-label').split('/')[0])
    )

len(title_lst), len(date_lst), len(stars_lst), len(user_lst)

(45, 45, 45, 45)

In [44]:
df = pd.DataFrame()

df['app'] = ['자란다' for i in range(len(title_lst))]
df['title'] = title_lst
df['user'] = user_lst
df['date'] = date_lst
df['stars'] = stars_lst
df['comment'] = review


print(df.shape)
print(df.dtypes)
df.head()

(45, 6)
app                object
title              object
user               object
date       datetime64[ns]
stars               int64
comment            object
dtype: object


,app,title,user,date,stars,comment
0,자란다,좋아요,maum.c,2019-10-31,5,원래 쓰고 있었는데 앱 나오니 훨씬 편하고 좋아요 나름 꽤 초창기부터 써왔는데 점점...
1,자란다,드디어 앱이 나왔군요~~!,파랑땡땡구리,2019-07-13,5,항상 감사하고 있습니다. 우리 애들 자란다 선생님 너무 좋아해요. 계속 자란다 선생...
2,자란다,선생님이 너무좋았답니다.,terry11117,2020-03-26,5,다른 돌봄회사들보다 체계적이어서 감동받았어요. 우선 자란다 선생님의 선택의 폭이 넓...
3,자란다,아이들이 자란다 선생님을 너무 좋아해요~~,ysmom23,2019-09-19,5,엄마는 힘들어서 못해주는 몸으로 놀아주기.. 자란쌤이 아이와 함께 놀아주니 아이가 ...
4,자란다,맞벌이 부부에게 큰 힘이 됩니다,별2개,2021-07-30,5,이런 리뷰를 잘 안쓰는데 자란다에게 고마운 마음을 표현하고자 끄적여봅니다^^\n\n...


# 3. 빅쿼리로 전송

## 3-1. 빅쿼리로 전송

In [35]:
from google.oauth2 import service_account
import pandas_gbq

In [46]:
cd = service_account.Credentials.from_service_account_file(
    '/Users/choeinbeom/Downloads/tidal-surf-302413-582aad3f1e46.json')
project_id = 'tidal-surf-302413'
destination_table = 'test_jaranda.reviews'
df.to_gbq(destination_table,project_id,if_exists='append',credentials=cd) 
print('migration complete')

1it [00:02,  2.76s/it]

migration complete


## 3-2. 빅쿼리 데이터 로드 및 확인

In [3]:
from google.cloud import bigquery

In [41]:
cd = service_account.Credentials.from_service_account_file(
    '/Users/choeinbeom/Downloads/tidal-surf-302413-582aad3f1e46.json')


client = bigquery.Client (project = 'tidal-surf-302413' , credentials = cd) 
query = 'SELECT * FROM test_jaranda.reviews' 

df_result = client.query(query).to_dataframe()

print(df_result.shape)
print(df_result.dtypes)

df_result.head()

(339, 6)
title                   object
user                    object
date       datetime64[ns, UTC]
stars                    int64
comment                 object
app                     object
dtype: object


,title,user,date,stars,comment,app
0,회원탈퇴,구름바이러스,2019-12-10 00:00:00+00:00,5,회원탈퇴 어떻게 하나요 선생님 지원하려다 앱 따로있는거 모르고 가입했는데,자란다
1,선생님들이 우수해요,nikiiiiaaaasss,2020-09-23 00:00:00+00:00,5,평균적으로 우수한 선생님이 많아요. 교사관리가\n잘 되고있는것 같아요,자란다
2,좋아요,maum.c,2019-10-31 00:00:00+00:00,5,원래 쓰고 있었는데 앱 나오니 훨씬 편하고 좋아요 나름 꽤 초창기부터 써왔는데 점점...,자란다
3,덕분에 아이도 저도 잘 놀아요,Ameri.cano,2019-10-31 00:00:00+00:00,5,"자란다 서비스를 쓴건 솔직히 아이를 위해서이기도 했지만, 저를 위해서 이기도 했어요...",자란다
4,자란다 앱 너무 편하네요,노수지,2019-10-31 00:00:00+00:00,5,예전에 웹으로 사용했었는데 정말 앱이 나와서 사용하기 편리해졌네요. 항상 느끼는거지...,자란다


In [43]:
crong = df_result[df_result['app']=='째깍악어']


In [26]:
newjrd = df_result[df_result['app']=='자란다'].sort_values(by = 'date', ascending=False)

newjrd[newjrd[['title', 'user']].duplicated()]

,title,user,date,stars,comment,app
40,선생님 관리가 제대로 안되는듯,nkein82,2021-06-08 00:00:00+00:00,5,예약하고 결재까지 다했는데 뜬금없이 그날은 안된다면서 하루전에 일방적으로 취소해달라...,자란다
46,신청서작성시 단계가 안 넘어가요!!!,사용하고싶어요!,2021-05-19 00:00:00+00:00,1,너무 화가나려하네요! 방문일정 등록하려는데 방문 종류 선택할때도 터치가 안먹는건지 ...,자란다
44,워킹맘은 절대 쓰지 마세요,VIP멤버펑,2021-03-31 00:00:00+00:00,1,일부러 추천율 100프로인 잘자란 선생님 골라서 (돌봄 시간도 100시간 넘는) 신...,자란다
33,너무 자주 어플이 먹통이 되어요ㅠㅠ,11166554443388,2021-03-17 00:00:00+00:00,5,자주 수업 리스트도 뜨지 않고 어플이 부팅되는데 너무 오래 걸려요ㅠㅠ,자란다
47,회원탈퇴 기능이 없네요,hhhhhhhh28472919,2021-01-22 00:00:00+00:00,1,너무 찜찜하고 신고하려다 말았는데 회원 탈퇴 기능 만들어주세요\n이런걸 리뷰에 써야...,자란다
24,ㅇㅇㅇㅇ,Icefall입니다,2020-11-27 00:00:00+00:00,5,카카오페이 등록 안됨 앱 느리고 오류 많음 ㅠ\n선생님 보다가 뒤로가기 안뎀,자란다
36,오늘 자란다의 일방적 “신청취소” 후 느낀점,사노리,2020-09-25 00:00:00+00:00,5,초1아들 영어 첫공부에 대한 블로그 추천글이 많아 학원보내기 불안한 시기에 적절히 ...,자란다
52,아이폰에서 최근에 계속 로딩만 뜨고 넘어가지를 않네요 ㅠㅠ 저만 그런건가요,저만안되나요,2020-08-04 00:00:00+00:00,2,앱 실행하면 첫 화면에서 로딩 아이콘만 뜨고 넘어가지를 못하네요. 잠깐 그런가보다 ...,자란다
32,탈퇴방법,ㅜㅏ나니미,2020-06-26 00:00:00+00:00,5,선생님 가입하려했는데 실수로 부모님으로 가입해서 다시 가입하려니까 이미 핸드폰번호가...,자란다
8,좋은 선생님 추천,해해해햇,2020-01-27 00:00:00+00:00,5,좋은 선생님이 너무 좋아요,자란다
